In [2]:
!pip install pot
!pip install torch
!pip install geomloss
!pip install wget
!pip install mdatagen
!pip install pot
!pip install torch
!pip install geomloss
!pip install wget
!pip install numpy scipy sklearn
!pip install jax
!pip install chex
!pip install attr
!pip install optax



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


...

  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [15 lines of output]
      The 'sklearn' PyPI package is deprecated, use 'scikit-learn'
      rather than 'sklearn' for pip commands.
      
      Here is how to fix this error in the main use cases:
      - use 'pip install scikit-learn' rather than 'pip install sklearn'
      - replace 'sklearn' by 'scikit-learn' in your pip requirements files
        (requirements.txt, setup.py, setup.cfg, Pipfile, etc ...)
      - if the 'sklearn' package is used by one of your dependencies,
        it would be great if you take some time to track which package uses
        'sklearn' instead of 'scikit-learn' and report it to their issue tracker
      - as a last resort, set the environment variable
        SKLEARN_ALLOW_DEPRECATED_SKLEARN_PACKAGE_INSTALL=True to avoid this error
      
      More information is available at
      https://github.com/scikit-learn/sklearn-


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import numpy as np
import torch
import torch.nn as nn
import pandas as pd

from geomloss import SamplesLoss

import ot

import os
import pickle as pkl
import copy

from sklearn.preprocessing import scale
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer

from utils import *
from SoftImpute import softimpute, cv_softimpute
#from DataSetLoader import dataset_loader, ground_truth
from SinkhornImputation import SinkhornImputation
from Sinkhorn_CMI import SinkhornImputation_CMI
from RR_imputer import RRimputer
import matplotlib.pyplot as plt

from Inject_Missing_Values import *

from Miss import *

import argparse
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)
logging.debug("test")
import pandas as pd

torch.set_default_tensor_type('torch.DoubleTensor')

c:\Users\zhossai3\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\__init__.py:1144: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\tensor\python_tensor.cpp:434.)
  _C._set_default_tensor_type(t)


In [4]:
import numpy as np
#from mbi import Domain, CliqueVector, Factor, LinearMeasurement
#from mbi import marginal_oracles, marginal_loss, synthetic_data
from typing import Callable
import jax
import jax.numpy as jnp
import chex
import attr
import optax

In [5]:
import pandas as pd

# Read CSV file into a Pandas DataFrame
df = pd.read_csv(r'C:\Users\zhossai3\Desktop\My code\OT_Fairness\Data\German Credit\German_Credit_Ground_Truth.csv', delimiter=',', header=0)

# Store feature columns in a DataFrame

X= df.iloc[:, :-1]  # Selects all rows and all columns except the last one
Y = df.iloc[:, -1]  # Selects all rows and only the last column

# Display the DataFrames
print(X)  # Print first few rows of features
 # Print first few rows of target variable
type(X)
groundTruth =   X


     checking-account  duration  credit-history  purpose  credit-amount  \
0                   1         6               0        7           1169   
1                   0        48               1        7           5951   
2                   3        12               0        4           2096   
3                   1        42               1        5           7882   
4                   1        24               2        1           4870   
..                ...       ...             ...      ...            ...   
995                 3        12               1        5           1736   
996                 1        30               1        2           3857   
997                 3        12               1        7            804   
998                 1        45               1        7           1845   
999                 0        45               0        2           4576   

     savings-account  employment-since  installment-rate  other-debtors  \
0                  4    

In [6]:
def conditional_mutual_information(data, X, Y, Z, delta=1):

  cmi = 0

  P_Z = data.groupby(Z).size()
  P_Z = P_Z / P_Z.sum()

  P_XZ = data.groupby(X + Z).size()
  P_XZ = P_XZ / P_XZ.sum()

  P_YZ = data.groupby(Y + Z).size()
  P_YZ = P_YZ / P_YZ.sum()

  P_XYZ = data.groupby(X + Y + Z).size()
  P_XYZ = P_XYZ / P_XYZ.sum()

  for ind in P_XYZ.index:
    x_ind = ind[:len(X)]
    y_ind = ind[len(X):len(X + Y)]
    z_ind = ind[len(X + Y):]

    xz_ind = x_ind + z_ind
    yz_ind = y_ind + z_ind
    xyz_ind = ind

    z_ind = pd.MultiIndex.from_tuples([z_ind], names=Z) if len(Z) != 1 else pd.Index(z_ind, name=Z[0])
    xz_ind = pd.MultiIndex.from_tuples([xz_ind], names=X + Z)
    yz_ind = pd.MultiIndex.from_tuples([yz_ind], names=Y + Z)
    xyz_ind = pd.MultiIndex.from_tuples([xyz_ind], names=X + Y + Z)

    cmi += delta * P_XYZ[xyz_ind].item() * np.log2(
      P_Z[z_ind].item() * P_XYZ[xyz_ind].item() / (P_XZ[xz_ind].item() * P_YZ[yz_ind].item()))

  return cmi

In [7]:
data = df # Convert tensor to numpy array
sensitive_attributes = [11, 19]

# Define different bin sizes for each sensitive attribute
bin_sizes = {
    11: 4,  # Binary attribute
    19: 2  # More categories
}


# Digitize multiple sensitive attributes with different bin sizes
#X_list = [
 #   np.digitize(data[:, col], bins=np.linspace(data[:, col].min(), data[:, col].max(), bin_sizes[col] + 1))
  #  for col in sensitive_attributes
#]
#x = np.column_stack(X_list)
#X = np.digitize(data[:,1],bins=np.linspace(data[:,1].min(), data[:,1].max(),1)) 
#y =np.digitize(Y,bins=np.linspace(Y.min(), Y.max(),4)) 
#z =  np.digitize(data[:, 7],bins=np.linspace(data[:, 7].min(), data[:, 7].max(), 4))

x = ['sex','age']
y = ['class-label']
z = ['credit-amount', 'job']


cmi_value =conditional_mutual_information(data, x, y, z, delta=1)
print("Conditional Mutual Information:", cmi_value)

Conditional Mutual Information: 0.027509775004326952


**30% MCAR**

In [8]:
generator = Inject_Missing_Values()

In [9]:
miss_mcar30,index_mcar30 = generator.MCAR(X,missing_rate=30)
type(miss_mcar30)

pandas.core.frame.DataFrame

In [10]:
miss_mcar30_numpy = miss_mcar30.to_numpy()
X_numpy = X.to_numpy()
Y_numpy = Y.to_numpy()

miss_mcar30_tensor = torch.tensor(miss_mcar30_numpy)
X_tensor = torch.tensor(X_numpy)
Y_tensor = torch.tensor(Y_numpy)

In [11]:
#sinkhorn
n_mcar30, d_mcar30 = miss_mcar30_tensor.shape
batchsize = 128 # If the batch size is larger than half the dataset's size,
                # it will be redefined in the imputation methods.
lr = 1e-2
#epsilon_mcar30 = pick_epsilon(data_mcar30)
epsilon_mcar30 = 0.3
print(epsilon_mcar30)


sk_imputer_mcar30 = SinkhornImputation_CMI(eps=epsilon_mcar30, batchsize=batchsize, lr=lr, niter=2000)
sk_imp_mcar30, sk_maes30mcar, sk_rmses30mcar,sk_loss_mcar30 = sk_imputer_mcar30.fit_transform(miss_mcar30_tensor, verbose=True, report_interval=50, X_true=X_tensor)
#using numpy version of data
#sk_imp_mcar30_numpy = sk_imp_mcar30.detach().cpu().numpy()

#sk_mae_mcar30 = MAE(sk_imp_mcar30_numpy,ground_truth_numpy ,mask_mcar30)
#sk_rmse_mcar30 = RMSE(sk_imp_mcar30_numpy, ground_truth_numpy,mask_mcar30)
#print(sk_mae_mcar30,sk_rmse_mcar30)

0.3


INFO:root:Iteration 0:	 Loss: 141462.4873	 Validation MAE: 101.7272	RMSE: 652.2069
INFO:root:Iteration 50:	 Loss: 232787.9238	 Validation MAE: 101.7068	RMSE: 652.2024
INFO:root:Iteration 100:	 Loss: 124714.8034	 Validation MAE: 101.7045	RMSE: 652.2028
INFO:root:Iteration 150:	 Loss: 126743.8962	 Validation MAE: 101.7024	RMSE: 652.2020
INFO:root:Iteration 200:	 Loss: 371995.4443	 Validation MAE: 101.7015	RMSE: 652.2014
INFO:root:Iteration 250:	 Loss: 233921.2374	 Validation MAE: 101.6988	RMSE: 652.2006
INFO:root:Iteration 300:	 Loss: 113255.1189	 Validation MAE: 101.6973	RMSE: 652.1991
INFO:root:Iteration 350:	 Loss: 172153.9468	 Validation MAE: 101.6953	RMSE: 652.1988
INFO:root:Iteration 400:	 Loss: 408844.0164	 Validation MAE: 101.6941	RMSE: 652.1995
INFO:root:Iteration 450:	 Loss: 174546.4256	 Validation MAE: 101.6906	RMSE: 652.2001
INFO:root:Iteration 500:	 Loss: 113432.9790	 Validation MAE: 101.6898	RMSE: 652.2006
INFO:root:Iteration 550:	 Loss: 441937.8325	 Validation MAE: 101.687

In [12]:
print(sk_imp_mcar30)

tensor([[ 1.2977,  6.0000,  0.9053,  ...,  1.0000,  1.0000,  0.0000],
        [ 0.0000, 48.0000,  1.0000,  ...,  1.0000,  0.8067,  0.0000],
        [ 3.0000, 12.0000,  0.0000,  ...,  0.9931,  1.0000,  0.0000],
        ...,
        [ 3.0000, 12.0000,  0.8682,  ...,  0.9894,  1.0000,  0.0000],
        [ 1.6850, 45.0000,  1.0000,  ...,  1.0000,  1.0000,  0.0000],
        [ 1.6568, 45.0000,  1.1246,  ...,  0.9603,  1.0000,  0.0000]],
       grad_fn=<IndexPutBackward0>)


In [13]:
X_list = [1,4,7]
Y_list = [9]
Z_list = [3]

In [19]:
import torch

import torch

def bucketize_torch(data, bins=10, method="uniform"):
    """
    Bucketizes continuous data into discrete bins using PyTorch.

    :param data: PyTorch tensor of shape (N, D)
    :param bins: Number of bins
    :param method: "uniform" (equal width) or "quantile" (equal frequency)
    :return: Discretized data (Tensor of bin indices)
    """
    # Ensure tensor is detached (no gradients)
    data = data.detach()

    # Initialize tensor for bucketized data
    bucketized_data = torch.zeros_like(data, dtype=torch.int64)

    for i in range(data.shape[1]):  # Loop over features
        feature = data[:, i]  # Extract column

        if method == "uniform":
            min_val, max_val = feature.min(), feature.max()
            bin_edges = torch.linspace(min_val, max_val, bins + 1)
        elif method == "quantile":
            bin_edges = torch.quantile(feature, torch.linspace(0, 1, bins + 1))

        # Apply bucketization
        bucketized_data[:, i] = torch.bucketize(feature, bin_edges) - 1  # Start bins from 0

    return bucketized_data
  # Bins start from 0

# Example usage
data_torch = sk_imp_mcar30  # 1000 samples, 10 continuous features
binned_data_torch = bucketize_torch(data_torch, bins=10,method="uniform")
print(binned_data_torch)


tensor([[ 4,  0,  2,  ...,  9,  9,  0],
        [-1,  6,  2,  ...,  9,  8,  0],
        [ 9,  1, -1,  ...,  9,  9,  0],
        ...,
        [ 9,  1,  2,  ...,  9,  9,  0],
        [ 5,  6,  2,  ...,  9,  9,  0],
        [ 5,  6,  2,  ...,  9,  9,  0]])


In [20]:
def compute_probabilities(data, vars_list):
    """ Compute probability distributions over given variables """
    unique, counts = torch.unique(data[:, vars_list], dim=0, return_counts=True)
    probs = counts.float() / data.shape[0]
    return unique, probs

def conditional_mutual_information_torch(data, X, Y, Z, delta=1):
    """
    Compute Conditional Mutual Information I(X;Y|Z) using PyTorch tensors.

    :param data: PyTorch tensor of shape (N, D) (discretized)
    :param X: List of column indices for variable X
    :param Y: List of column indices for variable Y
    :param Z: List of column indices for variable Z
    :param delta: Smoothing parameter to avoid log(0)
    :return: Conditional Mutual Information (scalar)
    """
    # Compute probability distributions
    unique_Z, P_Z = compute_probabilities(data, Z)
    unique_XZ, P_XZ = compute_probabilities(data, X + Z)
    unique_YZ, P_YZ = compute_probabilities(data, Y + Z)
    unique_XYZ, P_XYZ = compute_probabilities(data, X + Y + Z)

    # Compute CMI
    cmi = 0
    for i in range(len(unique_XYZ)):
        xyz = unique_XYZ[i]
        z = xyz[len(X + Y):]  # Extract Z values
        xz = torch.cat((xyz[:len(X)], z))  # Concatenate X and Z
        yz = torch.cat((xyz[len(X):len(X + Y)], z))  # Concatenate Y and Z

        # Ensure index matching
        mask_z = (unique_Z == z).all(dim=1)
        mask_xz = (unique_XZ == xz).all(dim=1)
        mask_yz = (unique_YZ == yz).all(dim=1)

        if mask_z.any() and mask_xz.any() and mask_yz.any():
            P_z = P_Z[mask_z].sum()
            P_xyz = P_XYZ[i]
            P_xz = P_XZ[mask_xz].sum()
            P_yz = P_YZ[mask_yz].sum()

            # Compute contribution to CMI (avoid log(0) by adding delta)
            cmi += delta * P_xyz * torch.log2((P_z * P_xyz) / (P_xz * P_yz + 1e-10))

    return cmi.item()  # Convert to Python scalar


In [21]:
# Generate synthetic dataset (continuous values)
data_torch = torch.randn(1000, 10)  # 1000 samples, 10 continuous features

# Define multiple feature sets
X = [0, 6, 9]  # Three features for X
Y = [2, 4]  # Two features for Y
Z = [3, 5, 7]  # Three features for Z

# Convert continuous data to discrete bins
binned_data = bucketize_torch(sk_imp_mcar30, bins=10, method="uniform")

# Compute Conditional Mutual Information (CMI) on discretized data
cmi_value = conditional_mutual_information_torch(binned_data, X, Y, Z)
print(f"Discrete CMI: {cmi_value:.4f}")


Discrete CMI: 1.6440
